# Galactic Assembly as Abstract Art

This is an interactive version of a visualisation that I originally made for understanding how galaxies assemble in the evolving Universe of the EAGLE simulations ([Schaye et al. 2015](https://ui.adsabs.harvard.edu/abs/2015MNRAS.446..521S/abstract), [Crain et al. 2015](https://ui.adsabs.harvard.edu/abs/2015MNRAS.450.1937C/abstract)). In the $\Lambda$-Cold Dark Matter ($\Lambda$-CDM) model, the current 'standard' cosmological model, galaxies assemble hierarchically, building their mass over time through the merging of smaller galactic building blocks. In the $\Lambda$-CDM model, this hierarchical assembly is mainly governed by the distribution of the cold dark matter in the very early universe, itself determined by the details of the events following the Big Bang. The EAGLE simulations simulate these events from just after the Big Bang, right out to the present day in a representative volume of a Universe. This visualisation shows the track of all of the simulation particles which eventually form into a galaxy with a similar mass, shape and size to our galaxy, the Milky Way - a relatively large disc galaxy. 

These events are commonly visualised through [movies](https://vimeo.com/93020498), which focus on the galaxy forming in the simulations, showing its formation at each simulation snapshot. By instead showing the position of all the particles which make a galaxy in one frame, we see a different side of galaxy evolution, exposing the importance of hierarchical assembly in forming the galaxies we observe in the Universe today. 

A bold streak of tracks through the center of the field belongs to the main progenitor of the final galaxy, formed from a large overdense region of gas and dark matter, early in the evolution of the simulation. The mass of this early progenitor acts as a magnet, sucking up gas and attracting smaller early galaxies with its gravitational pull. Meanwhile, other, large overdensities undergo the same process, becoming galaxies of nearly comparable size to the main galaxy. These too are pulled in, but are dragged in much more gradually, having gained angular momentum during their own formation. These loop around the main 'branch', forming coil-like patterns in the visualisation. Eventually, they merge onto the main galaxy, and they become one.

By toggling the dark matter particles in the visualisation, the fundamental role of this hidden component of our universe is laid bare. The Baryons (stars and gas) follow the dark matter to their fate as galaxies, and the exact properties of the dark matter affect the eventual outcome of the simulations. By using simulations like EAGLE to understand galaxy formation, we may be able to make predictions for the nature of dark matter which will aid in its detection and characterisation in the real Universe.

In [1]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from ipywidgets import interactive, FloatSlider, HBox, VBox, interact_manual
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'

In [2]:
with open('../EAGLE_MW_trace_coords.npy', 'rb') as f:
    coordinates = np.load(f)
    dmcoordinates = np.load(f)

In [3]:
downsample = 10
#mean = np.nanmean(coordinates[-1,:,:].reshape(np.shape(coordinates)[1],np.shape(coordinates)[2]), axis=0)
mean = np.nanmean(coordinates.reshape(np.shape(coordinates)[0]*np.shape(coordinates)[1],np.shape(coordinates)[2]), axis=0)
dmX = dmcoordinates[:,::downsample,0]-mean[0]
dmY = dmcoordinates[:,::downsample,1]-mean[1]
dmZ = dmcoordinates[:,::downsample,2]-mean[2]
X = coordinates[:,::downsample,0]-mean[0]
Y = coordinates[:,::downsample,1]-mean[1]
Z = coordinates[:,::downsample,2]-mean[2]

maxX, minX = np.nanmax([np.nanmax(dmX), np.nanmax(X)]), np.nanmin([np.nanmin(dmX), np.nanmin(X)])
maxY, minY = np.nanmax([np.nanmax(dmY), np.nanmax(Y)]), np.nanmin([np.nanmin(dmY), np.nanmin(Y)])
maxZ, minZ = np.nanmax([np.nanmax(dmZ), np.nanmax(Z)]), np.nanmin([np.nanmin(dmZ), np.nanmin(Z)])

max_range = np.array([maxX-minX, maxY-minY, maxZ-minZ]).max()
Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(maxX+minX)
Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(maxY+minY)
Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(maxZ+minZ)


In [4]:
def f(phi, theta, zoom, linewidths, baryons, darkmatter):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.view_init(phi, theta)
    fig.set_size_inches(10,10)
    
    if darkmatter:
        for i in range(np.shape(dmX)[1]): 
            ax.plot3D(dmX[:,i], dmY[:,i], dmZ[:,i], color='#7EAEBF', lw=2*linewidths, alpha=0.5)
    if baryons:
        for i in range(np.shape(X)[1]): 
            ax.plot3D(X[:,i], Y[:,i], Z[:,i], color='#FFFFFF', lw=linewidths, alpha=0.5)

    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')

    ax.set_axis_off()
    xlim, ylim, zlim = ax.get_xlim(), ax.get_ylim(), ax.get_zlim()
    ax.set_xlim(np.array(xlim)/zoom)
    ax.set_ylim(np.array(ylim)/zoom)
    ax.set_zlim(np.array(zlim)/zoom)
    ax.set_facecolor('Black')
    width= round(max_range/zoom/0.677,2)
    ax.text2D(0.05, -0.05, "Current Width = %s Mpc (%s MLy)" % (width, round((width)*3.262,2)), transform=ax.transAxes, color='Black', fontsize=15)
    plt.show()

In [5]:
interactive_plot = interactive(f, phi=widgets.IntSlider(min=-90, max=90, value=0, continuous_update=False, description='Pitch Angle'), 
                                  theta=widgets.IntSlider(min=0,max=360,value=0,continuous_update=False, description='Rotation'), 
                                  zoom=widgets.IntSlider(min=1,max=20, value=13,continuous_update=False, description='Zoom'), 
                                  linewidths=widgets.FloatLogSlider( value=0.02, base=10, min=-3,max=0., step=0.2, continuous_update=False, description='Line Width'),
                                  baryons=widgets.ToggleButton(value=True,description='Baryons',tooltip='Description',icon='check'),
                                  darkmatter=widgets.ToggleButton(value=False,description='Dark Matter',tooltip='Description',icon='check'))
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(IntSlider(value=0, continuous_update=False, description='Pitch Angle', max=90, min=-90),…

To rotate and pitch the image, you can use the Pitch Angle and Rotation sliders. The zoom slider allows you to change the extent of the box, but maintains the aspect ratio. The Line Width adjusts the width of the particle traces, allowing you to highlight lower or higher density regions (low line width allows you to see the densest regions, while a high line width highlights the very low density accreting gas and dark matter).

Finally, you can toggle the baryonic and dark matter particles on and off using the buttons. The default shows only the baryons, in white. The dark matter particles are shown as thicker blue lines - you can see that these form 'haloes' around the baryonic particles. The current width of the window is shown below the image in mega-parsecs and in millions of light years (the distances are technically in 'comoving' coordinates, which remain constant despite the expansion of the universe).